In [1]:
from keras.models import load_model
model = load_model('trained_model(epoch=6,batch_size=256).h5')
# model = load_model('trained_model_from_reviews(epoch=5,batch_size=128).h5')
print(model.summary())

2021-11-30 08:26:08.308608: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2021-11-30 08:26:08.308651: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-11-30 08:26:08.309100: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           4231168   
                                                                 
 spatial_dropout1d (SpatialD  (None, 200, 32)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 50)                16600     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 4,247,819
Trainable params: 4,247,819
Non-trainable params: 0
______________________________________________

In [2]:
import pandas as pd
#test_sets = <input data>  ==> string list 

# #data1

df = pd.read_csv("./Reviews.csv")
df = df[df['Score'] != 3]
df['sentiment'] = df['Score'].apply(lambda rating : 'positive' if rating > 3 else 'negative')
#df.head()
review_df1=df[['Text','sentiment']]
test_sets1=review_df1.Text.values
# sentiment_label = review_df.sentiment.factorize()


#data2
df = pd.read_csv("./Tweets.csv")
review_df=df[['text','airline_sentiment']]
review_df2 = review_df[review_df['airline_sentiment'] != 'neutral']
review_df2.columns=["Text","sentiment"]
test_sets2=review_df2.Text.values
# answers=review_df2.sentiment.values
# sentiment_label = review_df.airline_sentiment.factorize()

review_df=pd.concat([review_df1,review_df2],ignore_index=True)
print(review_df)
test_sets=review_df.Text.values
sentiment_label = review_df.sentiment.factorize()
answers=review_df.sentiment.values

length=len(test_sets1)

                                                     Text sentiment
0       I have bought several of the Vitality canned d...  positive
1       Product arrived labeled as Jumbo Salted Peanut...  negative
2       This is a confection that has been around a fe...  positive
3       If you are looking for the secret ingredient i...  negative
4       Great taffy at a great price.  There was a wid...  positive
...                                                   ...       ...
537350  @AmericanAir my flight was Cancelled Flightled...  negative
537351         @AmericanAir right on cue with the delays👌  negative
537352  @AmericanAir thank you we got on a different f...  positive
537353  @AmericanAir leaving over 20 minutes Late Flig...  negative
537354  @AmericanAir you have my money, you change my ...  negative

[537355 rows x 2 columns]


In [3]:
# predict function
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(test_sets)
# print(tokenizer.word_index)
encoded_docs = tokenizer.texts_to_sequences(test_sets)
# print(encoded_docs)
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    # print(tw)
    tw = pad_sequences(tw,maxlen=200)
    # print(tw)
    prediction = int(model.predict(tw).round().item())
#     print(model.predict(tw).item())
   # print("Predicted label: ", sentiment_label[1][prediction])
    return sentiment_label[1][prediction]

In [4]:
test_sentence1 = "I enjoyed my journey on this flight."
predict_sentiment(test_sentence1)

test_sentence2 = "This is the worst flight experience of my life!"
predict_sentiment(test_sentence2)

test_sentence3 = "This goods are bad"
predict_sentiment(test_sentence3)

test_sentence4="This goods are so so"
predict_sentiment(test_sentence4)

print(predict_sentiment(test_sentence1), predict_sentiment(test_sentence2), predict_sentiment(test_sentence3),predict_sentiment(test_sentence4))

positive negative negative positive


In [ ]:
#classification
import re
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud 
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
stopwords.update(["br", "href", "would"])

positives=dict()
negatives=dict()
pos_cnt=0
neg_cnt=0
cnt=0

meta_text=""
pos_meta_text=""
neg_meta_text=""
for i in range(length):
  text= re.sub('^@[a-zA-Z0-9]*','', test_sets1[i])
  words=text.split(' ')
  words=[word for word in words if word not in stopwords]
  text= ' '.join(words)
  text=''.join(list(filter(lambda x: x not in "!@#$%^&*()-='",text)))
  text = str(text).lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub(r'[^\w\s]','',text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  # print(text)
  result=predict_sentiment(text)
  words=text.split(' ')
  words = list(filter(None, words))
  words=[word for word in words if word not in stopwords]
  # print(words)
    

  text=" ".join(words)
  meta_text+=text+" "
    
    
  if result=='positive':
    pos_cnt+=1
    for word in words:
      if word in positives:
        positives[word]+=1
      else:
        positives[word]=1
    pos_meta_text+=text+" "
        
  

  else:
    neg_cnt+=1
    for word in words:
      if word in negatives:
        negatives[word]+=1
      else:
        negatives[word]=1
    neg_meta_text+=text+" "
        
  if pos_cnt%500==0 or neg_cnt%500==0:
    print(pos_cnt, neg_cnt)
  if result==answers[i]:
    cnt+=1
  else:
    print(i+1, result, answers[i], "%d/%d"%(cnt,i+1))
    
wordcloud = WordCloud(stopwords=stopwords).generate(meta_text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud_total.png', pad_inches=0.1)
plt.show()

# wordcloud = WordCloud(stopwords=stopwords).generate(pos_meta_text)
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.savefig('wordcloud_pos.png', pad_inches=0.1)
# plt.show()

# wordcloud = WordCloud(stopwords=stopwords).generate(neg_meta_text)
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.savefig('wordcloud_neg.png', pad_inches=0.1)
# plt.show()


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/2016314770/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1 0
4 positive negative 3/4
13 positive negative 11/13
17 positive negative 14/17
26 negative positive 22/26
48 positive negative 43/48
69 positive negative 63/69
101 positive negative 94/101
121 positive negative 113/121
141 positive negative 132/141
166 negative positive 156/166
191 positive negative 180/191
195 positive negative 183/195
206 positive negative 193/206
211 positive negative 197/211
235 positive negative 220/235
236 positive negative 220/236
239 positive negative 222/239
248 negative positive 230/248
254 negative positive 235/254
263 negative positive 243/263
300 negative positive 279/300
308 positive negative 286/308
359 positive negative 336/359
369 positive negative 345/369
420 negative positive 395/420
454 positive negative 428/454
465 positive negative 438/465
492 negative positive 464/492
509 positive negative 480/509
510 positive negative 480/510
513 positive negative 482/513
514 positive negative 482/514
533 positive negative 500/533
539 positive negative 505/53

4053 positive negative 3798/4053
4056 positive negative 3800/4056
4063 positive negative 3806/4063
4071 positive negative 3813/4071
4093 positive negative 3834/4093
4132 negative positive 3872/4132
4133 positive negative 3872/4133
4226 negative positive 3964/4226
4235 positive negative 3972/4235
4241 negative positive 3977/4241
4263 positive negative 3998/4263
4264 positive negative 3998/4264
4265 positive negative 3998/4265
4266 positive negative 3998/4266
4267 positive negative 3998/4267
4268 positive negative 3998/4268
4269 positive negative 3998/4269
4276 negative positive 4004/4276
4342 negative positive 4069/4342
4345 positive negative 4071/4345
4367 positive negative 4092/4367
4369 positive negative 4093/4369
4377 positive negative 4100/4377
4378 positive negative 4100/4378
4409 positive negative 4130/4409
4454 negative positive 4174/4454
4459 negative positive 4178/4459
4468 negative positive 4186/4468
4478 positive negative 4195/4478
4485 positive negative 4201/4485
4486 posit

7813 positive negative 7311/7813
7822 positive negative 7319/7822
7842 positive negative 7338/7842
7843 positive negative 7338/7843
7866 positive negative 7360/7866
7871 positive negative 7364/7871
7890 positive negative 7382/7890
7909 positive negative 7400/7909
7911 positive negative 7401/7911
7912 positive negative 7401/7912
7916 positive negative 7404/7916
7917 positive negative 7404/7917
7919 positive negative 7405/7919
7935 positive negative 7420/7935
7948 negative positive 7432/7948
7000 999
8009 positive negative 7492/8009
8010 positive negative 7492/8010
7023 1000
7024 1000
7025 1000
7026 1000
7027 1000
7028 1000
7029 1000
7030 1000
7031 1000
7032 1000
7033 1000
7034 1000
7035 1000
8035 positive negative 7516/8035
7036 1000
7037 1000
7038 1000
7039 1000
7040 1000
7041 1000
7042 1000
8052 positive negative 7532/8052
8061 positive negative 7540/8061
8083 positive negative 7561/8083
8143 negative positive 7620/8143
8150 positive negative 7626/8150
8154 positive negative 7629/8154

10158 1500
10159 1500
11682 positive negative 10952/11682
11696 positive negative 10965/11696
11697 positive negative 10965/11697
11698 positive negative 10965/11698
11705 positive negative 10971/11705
11707 positive negative 10972/11707
11711 positive negative 10975/11711
11723 positive negative 10986/11723
11763 positive negative 11025/11763
11774 positive negative 11035/11774
11794 positive negative 11054/11794
11834 positive negative 11093/11834
11877 positive negative 11135/11877
11894 positive negative 11151/11894
11897 negative positive 11153/11897
11943 positive negative 11198/11943
11952 positive negative 11206/11952
11971 positive negative 11224/11971
11978 positive negative 11230/11978
11983 negative positive 11234/11983
11988 negative positive 11238/11988
11992 negative positive 11241/11992
12007 positive negative 11255/12007
12032 positive negative 11279/12032
10500 1544
12103 positive negative 11349/12103
12128 positive negative 11373/12128
12129 positive negative 11373/1

...

459933 positive negative 432858/459933
459939 positive negative 432863/459939
459948 positive negative 432871/459948
459962 negative positive 432884/459962
459965 positive negative 432886/459965
459977 positive negative 432897/459977
459988 positive negative 432907/459988
459994 positive negative 432912/459994
460009 positive negative 432926/460009
460013 positive negative 432929/460013
460015 negative positive 432930/460015
460026 negative positive 432940/460026
460146 negative positive 433059/460146
460165 positive negative 433077/460165
460202 negative positive 433113/460202
460224 positive negative 433134/460224
404500 55725
460258 positive negative 433167/460258
460260 positive negative 433168/460260
460262 positive negative 433169/460262
460265 positive negative 433171/460265
460270 positive negative 433175/460270
460274 positive negative 433178/460274
460303 positive negative 433206/460303
460334 negative positive 433236/460334
460368 positive negative 433269/460368
460379 posit

407000 56035
463036 positive negative 435761/463036
463046 negative positive 435770/463046
463050 positive negative 435773/463050
463064 positive negative 435786/463064
463078 positive negative 435799/463078
463119 positive negative 435839/463119
463124 positive negative 435843/463124
463139 positive negative 435857/463139
463148 positive negative 435865/463148
463166 positive negative 435882/463166
463194 positive negative 435909/463194
463205 positive negative 435919/463205
463232 positive negative 435945/463232
463257 negative positive 435969/463257
463278 positive negative 435989/463278
463300 positive negative 436010/463300
463314 positive negative 436023/463314
463321 positive negative 436029/463321
463346 positive negative 436053/463346
463347 positive negative 436053/463347
463355 positive negative 436060/463355
463365 negative positive 436069/463365
463382 positive negative 436085/463382
463384 positive negative 436086/463384
463393 positive negative 436094/463393
463394 posit

465994 positive negative 438511/465994
466025 positive negative 438541/466025
466051 positive negative 438566/466051
466055 negative positive 438569/466055
466068 positive negative 438581/466068
466069 positive negative 438581/466069
466081 positive negative 438592/466081
466111 negative positive 438621/466111
466126 negative positive 438635/466126
466133 negative positive 438641/466133
466162 negative positive 438669/466162
466185 positive negative 438691/466185
466193 positive negative 438698/466193
466218 positive negative 438722/466218
466219 positive negative 438722/466219
466220 positive negative 438722/466220
466233 positive negative 438734/466233
466235 positive negative 438735/466235
466265 positive negative 438764/466265
466274 positive negative 438772/466274
466284 positive negative 438781/466284
466288 positive negative 438784/466288
466303 negative positive 438798/466303
466304 positive negative 438798/466304
466305 positive negative 438798/466305
466348 positive negative 

469214 positive negative 441522/469214
469253 positive negative 441560/469253
469265 positive negative 441571/469265
469266 positive negative 441571/469266
469267 positive negative 441571/469267
469269 positive negative 441572/469269
469271 positive negative 441573/469271
469272 positive negative 441573/469272
469320 positive negative 441620/469320
412500 56822
469323 positive negative 441622/469323
469341 negative positive 441639/469341
469346 positive negative 441643/469346
469354 negative positive 441650/469354
469369 positive negative 441664/469369
469381 positive negative 441675/469381
469382 positive negative 441675/469382
469402 positive negative 441694/469402
469403 positive negative 441694/469403
469409 positive negative 441699/469409
469411 positive negative 441700/469411
469416 negative positive 441704/469416
469418 positive negative 441705/469418
469430 positive negative 441716/469430
469432 positive negative 441717/469432
469433 positive negative 441717/469433
469434 posit

472298 positive negative 444398/472298
472313 positive negative 444412/472313
472314 negative positive 444412/472314
472326 negative positive 444423/472326
472347 positive negative 444443/472347
472349 negative positive 444444/472349
472377 negative positive 444471/472377
472398 negative positive 444491/472398
472408 positive negative 444500/472408
472411 positive negative 444502/472411
472425 negative positive 444515/472425
472438 positive negative 444527/472438
472461 positive negative 444549/472461
472476 positive negative 444563/472476
472482 positive negative 444568/472482
472498 negative positive 444583/472498
472615 negative positive 444699/472615
472628 positive negative 444711/472628
472655 positive negative 444737/472655
472664 positive negative 444745/472664
472684 positive negative 444764/472684
472685 positive negative 444764/472685
472708 positive negative 444786/472708
472713 positive negative 444790/472713
472732 positive negative 444808/472732
472751 positive negative 

In [ ]:
wordcloud = WordCloud(stopwords=stopwords).generate(meta_text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud_total.png', pad_inches=0.1)
plt.show()


In [ ]:
import operator


# print(dict(positives))
positives=dict(positives)
negatives=dict(negatives)

sPositives= sorted(positives.items(), key=operator.itemgetter(1),reverse=True)
sNegatives= sorted(negatives.items(), key=operator.itemgetter(1),reverse=True)

sPositives=dict(sPositives)
sNegatives=dict(sNegatives)
pos_keys=list(sPositives.keys())
pos_values=list(sPositives.values())
neg_keys=list(sNegatives.keys())
neg_values=list(sNegatives.values())

top_pos_keys=pos_keys[:80]
top_pos_cnt=pos_values[:80]
top_neg_keys=neg_keys[:80]
top_neg_cnt=neg_values[:80]


# print(top_pos_keys)
cnt=0
i=0
while i<len(top_pos_keys):
    if top_pos_keys[i] in top_neg_keys:
        
        j=top_neg_keys.index(top_pos_keys[i])
        del top_pos_keys[i]
        del top_pos_cnt[i]
        del top_neg_keys[j]
        del top_neg_cnt[j]
    
    else:
        i+=1
print(top_pos_keys)
# print(top_neg_keys)
    

In [ ]:
wordcloud = WordCloud(stopwords=stopwords).generate(' '.join(top_pos_keys))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud_pos.png', pad_inches=0.1)
plt.show()

wordcloud = WordCloud(stopwords=stopwords).generate(' '.join(top_neg_keys))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud_neg.png', pad_inches=0.1)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

X=['positive','negative']
freq=[pos_cnt,neg_cnt]

plt.bar(X,freq, color=['limegreen','red'])
plt.savefig('./count.png', dpi=200, bbox_inches='tight', pad_inches=0.1)
plt.show()

top_pos_keys=top_pos_keys[:20]
top_pos_cnt=top_pos_cnt[:20]

top_neg_keys=top_neg_keys[:20]
top_neg_cnt=top_neg_cnt[:20]

# most top frequent 20 positive words
colors = sns.color_palette('YlGn',20)
plt.barh(top_pos_keys[::-1], top_pos_cnt[::-1], color=colors)
plt.savefig('./positive.png', dpi=200, bbox_inches='tight', pad_inches=0.1)
plt.show()

# most top frequent 20 negative words
colors = sns.color_palette('Reds',20)
plt.barh(top_neg_keys[::-1], top_neg_cnt[::-1], color=colors)
plt.savefig('./negative.png', dpi=200, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
from docx import Document
from docx.shared import Inches
from docx.shared import Pt

document = Document() 
document.add_heading('Sentiment Analysis Result', level = 0)
p = document.add_paragraph(style='ListNumber')
p.add_run().font.size=Pt(12)
p.add_run('Classification').bold=True
document.add_picture('count.png', width=Inches(5.5))
p = document.add_paragraph("The number of positive responses: "+str(pos_cnt)+ " ("+str(round(pos_cnt*100/(pos_cnt+neg_cnt),2))+"%)", style='ListBullet3')
p = document.add_paragraph("The number of negative responses: "+str(neg_cnt)+ " ("+str(round(neg_cnt*100/(pos_cnt+neg_cnt),2))+"%)", style='ListBullet3')
document.add_picture('wordcloud_total.png', width=Inches(5))
# document.add_page_break()

p = document.add_paragraph(style='ListNumber') 
p.add_run('Positive').bold=True
p = document.add_paragraph("The number of positive response: "+str(pos_cnt), style='ListBullet2')
p = document.add_paragraph("Top 20 frequent words: ", style='ListBullet2')
document.add_picture('positive.png', width=Inches(5.5))

table = document.add_table(rows = 21, cols = 2, style = "Table Grid")
for i in range(21):
  table.cell(i,0).width = Inches(1.25)
  table.cell(i,1).width = Inches(1.25)
table.rows[0].cells[0].text = "word"
table.rows[0].cells[1].text = "count"
for i in range(20):
  table.rows[i+1].cells[0].text = top_pos_keys[i]
  table.rows[i+1].cells[1].text = str(top_pos_cnt[i])
document.add_picture('wordcloud_pos.png', width=Inches(5))
document.add_page_break()

p = document.add_paragraph(style='ListNumber')
p.add_run('Negative').bold=True
p = document.add_paragraph("The number of negative response: "+str(neg_cnt), style='ListBullet2')
p = document.add_paragraph("Top 20 frequent words: ", style='ListBullet2')
document.add_picture('negative.png', width=Inches(5.5))

table = document.add_table(rows = 21, cols = 2, style = "Table Grid")
for i in range(21):
  table.cell(i,0).width = Inches(1.25)
  table.cell(i,1).width = Inches(1.25)
table.rows[0].cells[0].text = "word"
table.rows[0].cells[1].text = "count"

for i in range(20):
#   if len(neg_keys)<=i:
#     continue
  table.rows[i+1].cells[0].text = top_neg_keys[i]
  table.rows[i+1].cells[1].text = str(top_neg_cnt[i])
document.add_picture('wordcloud_neg.png', width=Inches(5))

document.save("result.docx")